# Import Thư Viện và Code Xử Lý

In [1]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz # Thư viện so sánh chuỗi

# ****************************************Format**********************************************

from deep_translator import GoogleTranslator # Thư viện dịch văn bản

def translator(text):
    """
    Dịch văn bản từ ngôn ngữ hiện tại sang tiếng Anh.
    # Đối với các trang quốc tế: Amazon, GadgetsNow, Reliance Digital
    """
    
    type_trans = GoogleTranslator(source='auto', target='en')
    trans = type_trans.translate(text)
    return trans

def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s

def convert(a):
    b = a.replace(" ", '')
    c = b.replace("INR", '')
    d = c.replace(",", '')
    d = d.replace("`", '')
    f = d.replace("₹", '')
    f1 = f.replace(".", '')
    f2 = f1.replace("₫", '')
    g = int(float(f2))
    return g

def format_price(price):
    """Formats a price number into a string with commas, currency symbol, and handles potential decimals.

  Args:
      price: The price as a number (float or int).

  Returns:
      A string representing the formatted price.
  """
    # Handle potential decimals
    if isinstance(price, float):
        price_str = "{:,.2f}".format(price)  # Two decimal places for floats
    else:
        price_str = "{:,}".format(price)  # No decimal places for integers

    return price_str + "đ"  # Add currency symbol (đ)
# **********************************************************************************************


C:\Users\truon\AppData\Local\Temp\ipykernel_16052\2412148540.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
d:\anaconda3\envs\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Crawl dữ liệu từ trang web

## Điện máy xanh

In [8]:


# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        global dienmayxanh
        #name=no_accent_vietnamese(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        dienmayxanh = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'
        dienmayxanh_link = dienmayxanh
        res = requests.get(
            f'https://www.dienmayxanh.com/tim-kiem?key={name2}', headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(soup)
        dienmayxanh_page = soup.select('a.main-contain')
        #print(dienmayxanh_page)
        dienmayxanh_page_length = int(len(dienmayxanh_page))
        # print(dienmayxanh_page_length)
        for i in range(0, dienmayxanh_page_length):
            name1 = name.upper()
            dienmayxanh_name = soup.select(
                'a.main-contain>h3')[i].getText().strip().upper()
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            if similarity_score > 5:
                dienmayxanh_name = soup.select(
                    'a.main-contain>h3')[i].getText().strip()       
                
                dienmayxanh_images = soup.select('a.main-contain')
                
                # Kiểm tra xem có ảnh được trả về không
                if dienmayxanh_images:
                    # Nếu có ảnh, thực hiện lấy đường link ảnh
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[0].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[0].find_all('img', class_='lazyload')[0]['data-src']
                    else:
                        # Nếu không có cả 'src' và 'data-src', gán một giá trị mặc định
                        dienmayxanh_image = '0'
                else:
                    # Nếu không có ảnh, gán một giá trị mặc định
                    dienmayxanh_image = '0'
                                
                dienmayxanh_price = soup.select(
                    'a.main-contain>strong.price')[i].getText().strip().upper()
                dienmayxanh_price=dienmayxanh_price.strip('₫')
                print("Điện Máy Xanh:")
                print("Tên Sản Phẩm:", dienmayxanh_name)
                print("Giá:", dienmayxanh_price)
                print("Link Ảnh:", dienmayxanh_image)
                print("Link:", dienmayxanh_link)
                
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == dienmayxanh_page_length:
                    dienmayxanh_price = '0'
                    print("Điện Máy Xanh : No product found!")
                    print("-----------------------------")
                    break

        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_link
    except:
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_link = '0'
        dienmayxanh_image = '0'
    return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_link

# ******************************************************************************************************

# Test
dienmayxanh("Điện thoai")


Searching in Điện Máy Xanh...
Điện Máy Xanh:
Tên Sản Phẩm: iPhone 15 Pro Max
Giá: 29.590.000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg
Link: https://www.dienmayxanh.com/tim-kiem?key=Điện+thoai
---------------------------------


('29.590.000',
 'iPhone 15 Pro Max',
 'https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg',
 'https://www.dienmayxanh.com/tim-kiem?key=Điện+thoai')

## Thế giới di động

In [7]:
# ****************************************Thế giới di động**********************************************
def tgdd(name):
    try:
        global tgdd
        #name=no_accent_vietnamese(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        tgdd = f'https://www.thegioididong.com/tim-kiem?key={name2}'
        tgdd_link = tgdd
        res = requests.get(
            f'https://www.thegioididong.com/tim-kiem?key={name2}', headers=headers)
        print("\nSearching in Thế giới di động...")
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(soup)
        tgdd_page = soup.select('a.main-contain')
        #print(tgdd_page)
        tgdd_page_length = int(len(tgdd_page))
        print(tgdd_page_length)
        for i in range(0, tgdd_page_length):
            name1 = name.upper()
            tgdd_name = soup.select(
                'a.main-contain>h3')[i].getText().strip().upper()
            similarity_score = fuzz.ratio(name.upper(), tgdd_name)
            if similarity_score > 5:
                tgdd_name = soup.select(
                    'a.main-contain>h3')[i].getText().strip()       
                
                tgdd_images = soup.select('a.main-contain')
                
                # Kiểm tra xem có ảnh được trả về không
                if tgdd_images:
                    # Nếu có ảnh, thực hiện lấy đường link ảnh
                    if tgdd_images[i].find('img').get('src'):
                        tgdd_image = tgdd_images[i].find('img')['src']
                    elif tgdd_images[0].find_all('img', class_='lazyload'):
                        tgdd_image = tgdd_images[0].find_all('img', class_='lazyload')[0]['data-src']
                    else:
                        # Nếu không có cả 'src' và 'data-src', gán một giá trị mặc định
                        tgdd_image = '0'
                else:
                    # Nếu không có ảnh, gán một giá trị mặc định
                    tgdd_image = '0'
                                
                tgdd_price = soup.select(
                    'a.main-contain>strong.price')[i].getText().strip().upper()
                tgdd_price=tgdd_price.strip('₫')
                print("Thế giới di động:")
                print("Tên Sản Phẩm:", tgdd_name)
                print("Giá:", tgdd_price)
                print("Link Ảnh:", tgdd_image)
                print("Link:", tgdd_link) 
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == tgdd_page_length:
                    tgdd_price = '0'
                    print("Thế giới di động : No product found!")
                    print("-----------------------------")
                    break

        return tgdd_price, tgdd_name[0:50], tgdd_image, tgdd_link
    except:
        print("Thế giới di động: No product found!1")
        print("---------------------------------")
        tgdd_price = '0'
        tgdd_name = '0'
        tgdd_link = '0'
        tgdd_image = '0'
    return tgdd_price, tgdd_name[0:50], tgdd_image, tgdd_link
# **********************************************************************************************

# Test
tgdd("Điện thoại")


Searching in Thế giới di động...
20
Thế giới di động:
Tên Sản Phẩm: iPhone 15 Pro Max
Giá: 29.590.000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg
Link: https://www.thegioididong.com/tim-kiem?key=Điện+thoại
---------------------------------


('29.590.000',
 'iPhone 15 Pro Max',
 'https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg',
 'https://www.thegioididong.com/tim-kiem?key=Điện+thoại')

## Amazon

In [9]:
# # ****************************************Amazon**********************************************
def amazon(name):
    try:
        global amazon
        name = translator(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        amazon = f'https://www.amazon.in/{name1}/s?k={name2}'
        amazon_link = amazon
        res = requests.get(
            f'https://www.amazon.in/{name1}/s?k={name2}', headers=headers)
        print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text, 'html.parser')
        amazon_page = soup.select('.a-color-base.a-text-normal')
        amazon_page_length = int(len(amazon_page))
        for i in range(0, amazon_page_length):
            name = name.upper()
            amazon_name = soup.select(
                '.a-color-base.a-text-normal')[i].getText().strip().upper()
            
            similarity_score = fuzz.ratio(name.upper(), amazon_name)    
        
            # if name in amazon_name:
            if similarity_score > 5:
                amazon_name = soup.select(
                    '.a-color-base.a-text-normal')[i].getText().strip()
                amazon_images = soup.select(
                    '.a-section.aok-relative.s-image-fixed-height')
                amazon_image = amazon_images[0].find_all(
                    'img', class_='s-image')[0]
                amazon_image = amazon_image['src']
                amazon_price = soup.select(
                    '.a-price-whole')[i].getText().strip().upper()
                print("Amazon:")
                print("Tên Sản Phẩm:", amazon_name)
                print("Giá:", amazon_price)
                print("Link Ảnh:", amazon_image)
                print("Link:", amazon_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == amazon_page_length:
                    amazon_price = '0'
                    print("amazon : No product found!")
                    print("-----------------------------")
                    break

        return amazon_price, amazon_name[0:50], amazon_image, amazon_link
    except:
        print("Amazon: No product found!")
        print("---------------------------------")
        amazon_price = '0'
        amazon_name = '0'
        amazon_link = '0'
        amazon_image = '0'
    return amazon_price, amazon_name[0:50], amazon_image, amazon_link

# **********************************************************************************************
# Test
amazon("Điện thoại")


Searching in amazon...
Amazon:
Tên Sản Phẩm: Apple iPhone 13 (128GB) - Midnight
Giá: 48,499
Link Ảnh: https://m.media-amazon.com/images/I/61VuVU94RnL._AC_UY218_.jpg
Link: https://www.amazon.in/Phone/s?k=Phone
---------------------------------


('48,499',
 'Apple iPhone 13 (128GB) - Midnight',
 'https://m.media-amazon.com/images/I/61VuVU94RnL._AC_UY218_.jpg',
 'https://www.amazon.in/Phone/s?k=Phone')